# PydanticOuputParser

`PydanticOutputParser` 는 언어 모델의 출력을 더 구조화된 정보로 변환하는 데 도움이 되는 클래스입니다. 언어 모델에서 나온 단순 텍스트를 넘어서, 사용자가 필요로 하는 정보를 명확하고 체계적인 형태로 제공할 수 있습니다.

이 클래스를 활용함으로써, 언어 모델의 출력을 특정 데이터 모델에 맞게 변환하여, 더 용이하게 정보를 처리하고 활용할 수 있게 됩니다.

PydanticOutputParser에는 주로 두 가지 핵심 메서드가 구현되어야 합니다.

- `get_format_instructions()`: 이 메서드는 언어 모델이 출력해야 할 정보의 형식을 정의하는 지침을 제공합니다. 예를 들어, 언어 모델이 출력해야 할 데이터의 필드와 그 형태를 설명하는 지침을 문자열로 반환할 수 있습니다. 이 지침은 사용자가 원하는 출력 형식을 정확히 이해하고 준비하는 데 도움을 줍니다.
- `parse()`: 이 메서드는 언어 모델의 출력(문자열로 가정)을 받아들여 이를 특정 구조로 분석하고 변환합니다. Pydantic와 같은 도구를 사용하여, 입력된 문자열을 사전 정의된 스키마에 따라 검증하고, 해당 스키마를 따르는 데이터 구조로 변환합니다. 이 과정을 통해 언어 모델의 출력을 더 사용하기 쉬운 형태로 가공할 수 있습니다.


In [1]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field


llm = ChatOpenAI(temperature=0)

다음은 이메일 본문 예시입니다.


In [2]:
email_conversation = """From: 김철수 (chulsoo.kim@bikecorporation.me)
To: 이은채 (eunchae@teddyinternational.me)
Subject: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안

안녕하세요, 이은채 대리님,

저는 바이크코퍼레이션의 김철수 상무입니다. 최근 보도자료를 통해 귀사의 신규 자전거 "ZENESIS"에 대해 알게 되었습니다. 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 혁신과 품질을 선도하는 기업으로, 이 분야에서의 장기적인 경험과 전문성을 가지고 있습니다.

ZENESIS 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 배터리 성능, 그리고 디자인 측면에 대한 정보가 필요합니다. 이를 통해 저희가 제안할 유통 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.

또한, 협력 가능성을 더 깊이 논의하기 위해 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안합니다. 귀사 사무실에서 만나 이야기를 나눌 수 있을까요?

감사합니다.

김철수
상무이사
바이크코퍼레이션
"""

위와 같은 이메일 내용이 주어졌을 때 아래의 PyDantic 스타일로 정의된 클래스를 사용하여 이메일의 정보를 파싱해 보겠습니다.


In [3]:
class EmailSummary(BaseModel):
    person: str = Field(description="메일을 보낸 사람")
    email: str = Field(description="메일을 보낸 사람의 이메일 주소")
    subject: str = Field(description="메일 제목")
    summary: str = Field(description="메일 본문을 요약한 텍스트")
    date: str = Field(description="메일 본문에 언급된 미팅 날짜와 시간")


# PydanticOutputParser 생성
parser = PydanticOutputParser(pydantic_object=EmailSummary)

프롬프트를 정의합니다.

1. `question`: 유저의 질문을 받습니다.
2. `email_conversation`: 이메일 본문의 내용을 입력합니다.
3. `format`: 형식을 지정합니다.


In [4]:
prompt = PromptTemplate.from_template(
    """
You are a helpful assistant. Please answer the following questions in KOREAN.

QUESTION:
{question}

EMAIL CONVERSATION:
{email_conversation}

FORMAT:
{format}
"""
)

# format 에 PydanticOutputParser의 format을 추가
prompt = prompt.partial(format=parser.get_format_instructions())

다음으로는 Chain 을 생성합니다.


In [5]:
chain = prompt | llm

체인을 실행하고 결과를 확인합니다.


In [6]:
# chain 을 실행하고 결과를 출력합니다.
response = chain.invoke(
    {
        "email_conversation": email_conversation,
        "question": "이메일 내용중 주요 내용을 추출해 주세요.",
    }
)

# 결과는 JSON 형태로 출력됩니다.
print(response.content)

{
  "properties": {
    "person": {
      "title": "Person",
      "description": "이메일을 발신한 사람",
      "type": "string"
    },
    "email": {
      "title": "Email",
      "description": "이메일을 발신한 사람의 이메일 주소",
      "type": "string"
    },
    "subject": {
      "title": "Subject",
      "description": "이메일 제목",
      "type": "string"
    },
    "summary": {
      "title": "Summary",
      "description": "이메일 본문을 요약한 설명",
      "type": "string"
    },
    "date": {
      "title": "Date",
      "description": "이메일 본문에 포함된 날짜와 시간",
      "type": "string"
    }
  },
  "required": ["person", "email", "subject", "summary", "date"]
}


마지막으로 `parser`를 사용하여 결과를 파싱하고 `EmailSummary` 객체로 변환합니다.


In [8]:
parser.parse(response.content)

OutputParserException: Failed to parse EmailSummary from completion {"properties": {"person": {"title": "Person", "description": "\uc774\uba54\uc77c\uc744 \ubc1c\uc2e0\ud55c \uc0ac\ub78c", "type": "string"}, "email": {"title": "Email", "description": "\uc774\uba54\uc77c\uc744 \ubc1c\uc2e0\ud55c \uc0ac\ub78c\uc758 \uc774\uba54\uc77c \uc8fc\uc18c", "type": "string"}, "subject": {"title": "Subject", "description": "\uc774\uba54\uc77c \uc81c\ubaa9", "type": "string"}, "summary": {"title": "Summary", "description": "\uc774\uba54\uc77c \ubcf8\ubb38\uc744 \uc694\uc57d\ud55c \uc124\uba85", "type": "string"}, "date": {"title": "Date", "description": "\uc774\uba54\uc77c \ubcf8\ubb38\uc5d0 \ud3ec\ud568\ub41c \ub0a0\uc9dc\uc640 \uc2dc\uac04", "type": "string"}}, "required": ["person", "email", "subject", "summary", "date"]}. Got: 5 validation errors for EmailSummary
person
  field required (type=value_error.missing)
email
  field required (type=value_error.missing)
subject
  field required (type=v

## parser 가 결합된 체인 생성

출력 결과를 객체로 생성할 수 있습니다.


In [9]:
# 출력 파서를 추가하여 전체 체인을 재구성합니다.
chain = prompt | llm | parser

In [10]:
# chain 을 실행하고 결과를 출력합니다.
response = chain.invoke(
    {
        "email_conversation": email_conversation,
        "question": "이메일 내용중 주요 내용을 추출해 주세요.",
    }
)

# 결과는 EmailSummary 객체 형태로 출력됩니다.
response

OutputParserException: Failed to parse EmailSummary from completion {"properties": {"person": {"title": "Person", "description": "\uc774\uba54\uc77c\uc744 \ubc1c\uc2e0\ud55c \uc0ac\ub78c", "type": "string"}, "email": {"title": "Email", "description": "\uc774\uba54\uc77c\uc744 \ubc1c\uc2e0\ud55c \uc0ac\ub78c\uc758 \uc774\uba54\uc77c \uc8fc\uc18c", "type": "string"}, "subject": {"title": "Subject", "description": "\uc774\uba54\uc77c \uc81c\ubaa9", "type": "string"}, "summary": {"title": "Summary", "description": "\uc774\uba54\uc77c \ubcf8\ubb38\uc744 \uc694\uc57d\ud55c \ubb38\uc790\uc5f4", "type": "string"}, "date": {"title": "Date", "description": "\uc774\uba54\uc77c \ubcf8\ubb38\uc5d0 \ud3ec\ud568\ub41c \ub0a0\uc9dc\uc640 \uc2dc\uac04", "type": "string"}}, "required": ["person", "email", "subject", "summary", "date"]}. Got: 5 validation errors for EmailSummary
person
  field required (type=value_error.missing)
email
  field required (type=value_error.missing)
subject
  field required (